In [1]:
import boto3
from configparser import ConfigParser
import base64
import pandas as pd
import json

In [11]:
config_parser = ConfigParser()
config_parser.read(["credentials.cfg", "aws.cfg"])

['credentials.cfg', 'aws.cfg']

In [12]:
ACCESS_KEY =  config_parser.get("AWS_IAM_USER", "KEY") # Iam user access key.
SECRET_KEY = config_parser.get("AWS_IAM_USER", "SECRET") # IAM user secret key
IAM_USER_REGION = config_parser.get("AWS_IAM_USER", "REGION")
REDSHIFT_USER = config_parser.get("REDSHIFT_USER", "USER")
REDSHIFT_PASSWORD = config_parser.get("REDSHIFT_USER", "PASSWORD")
REDSHIFT_IAM_ROLE_NAME = config_parser.get("REDSHIFT_CLUSTER", "IAM_ROLE_NAME")
REDSHIFT_NAMESPACE_NAME = config_parser.get("REDSHIFT_CLUSTER", "NAMESPACE_NAME")

pd.DataFrame({
    "Configurations": [
        "ACCESS KEY", "SECRET KEY", "IAM REGION", "REDSHIFT IAM ROLE",
        "REDSHIFT USER", "REDSHIFT PASSWORD", "REDSHIFT NAMESPACE"
    ],
    "Values": [
        ACCESS_KEY, SECRET_KEY, IAM_USER_REGION, REDSHIFT_IAM_ROLE_NAME,
        REDSHIFT_USER, REDSHIFT_PASSWORD, REDSHIFT_NAMESPACE_NAME
    ]
})

,Configurations,Values
0,ACCESS KEY,QUtJQVpDQVdOT1JHVEVUQ1VGRkw=
1,SECRET KEY,bzVlRUhLcm95U0tJZFRWeXJ4aHRWMGQrZXhDeWZQRmlGSW...
2,IAM REGION,us-east-1
3,REDSHIFT IAM ROLE,redshift-role
4,REDSHIFT USER,admin
5,REDSHIFT PASSWORD,YWRtaW4=
6,REDSHIFT NAMESPACE,default


In [4]:
s3 = boto3.resource("s3", region_name=IAM_USER_REGION,
                    aws_access_key_id=base64.b64decode(ACCESS_KEY).decode('utf-8'),
                    aws_secret_access_key=base64.b64decode(SECRET_KEY).decode('utf-8'))
iam = boto3.client('iam', region_name=IAM_USER_REGION,
                   aws_access_key_id=base64.b64decode(ACCESS_KEY).decode('utf-8'),
                   aws_secret_access_key=base64.b64decode(SECRET_KEY).decode('utf-8'))
ec2 = boto3.resource("ec2", region_name=IAM_USER_REGION,
                     aws_access_key_id=base64.b64decode(ACCESS_KEY).decode('utf-8'),
                     aws_secret_access_key=base64.b64decode(SECRET_KEY).decode('utf-8'))
redshift = boto3.client('redshift-serverless', region_name=IAM_USER_REGION,
                        aws_access_key_id=base64.b64decode(ACCESS_KEY).decode('utf-8'),
                        aws_secret_access_key=base64.b64decode(SECRET_KEY).decode('utf-8'))

In [ ]:
bucket=s3.Bucket('quannct-sparkify')
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/')] # Load song data files

print ("Song data files: ")
print(song_data_files)

log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log-data')] # Load log data files

print(log_data_files)

In [38]:
try:
    iam_role = iam.create_role(
        Path="/",
        RoleName=REDSHIFT_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
                            'Effect': 'Allow',
                            'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'}
        )
    )
    print(iam_role)
except Exception as exception:
    print(exception)

{'Role': {'Path': '/', 'RoleName': 'redshift-role', 'RoleId': 'AROAZCAWNORGVJ7VDMU42', 'Arn': 'arn:aws:iam::622817277005:role/redshift-role', 'CreateDate': datetime.datetime(2023, 4, 15, 17, 40, 46, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Statement': [{'Action': 'sts:AssumeRole', 'Effect': 'Allow', 'Principal': {'Service': 'redshift.amazonaws.com'}}], 'Version': '2012-10-17'}}, 'ResponseMetadata': {'RequestId': '70825cce-4e7f-4a67-b64d-ea0654d16172', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '70825cce-4e7f-4a67-b64d-ea0654d16172', 'content-type': 'text/xml', 'content-length': '782', 'date': 'Sat, 15 Apr 2023 17:40:46 GMT'}, 'RetryAttempts': 0}}


In [5]:
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=REDSHIFT_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess"
                       )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [5]:
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=REDSHIFT_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::622817277005:role/redshift-role


In [10]:
vpc = ec2.Vpc(id="vpc-0e34785bb4ea2396b")
defaultSg = list(vpc.security_groups.all())[0]
subnets = list(vpc.subnets.all())

print (defaultSg)
for subnet in subnets:
    print(subnet)

ec2.SecurityGroup(id='sg-041b5e77796aff823')
[ec2.Subnet(id='subnet-00e0cb324cf25d907'), ec2.Subnet(id='subnet-0f7a7fa0a5bf2adb2'), ec2.Subnet(id='subnet-0fb5311d8e5dacdba'), ec2.Subnet(id='subnet-06dc4f89d43d295b8'), ec2.Subnet(id='subnet-043e3e78b54cc5458'), ec2.Subnet(id='subnet-02aa5315993b0e9b7')]
ec2.Subnet(id='subnet-00e0cb324cf25d907')
ec2.Subnet(id='subnet-0f7a7fa0a5bf2adb2')
ec2.Subnet(id='subnet-0fb5311d8e5dacdba')
ec2.Subnet(id='subnet-06dc4f89d43d295b8')
ec2.Subnet(id='subnet-043e3e78b54cc5458')
ec2.Subnet(id='subnet-02aa5315993b0e9b7')


In [16]:
try:
    redshift_response = redshift.create_namespace(
        adminUserPassword=base64.b64decode(REDSHIFT_PASSWORD).decode('utf-8'),
        adminUsername=REDSHIFT_USER,
        iamRoles=[
            "arn:aws:iam::622817277005:role/redshift-role",
        ],
        namespaceName=REDSHIFT_NAMESPACE_NAME
    )
except Exception as e:
    print(e)

An error occurred (ValidationException) when calling the CreateNamespace operation: The IAM Role ARN: redshift-role is invalid.
